In [ ]:
!pip3 install -U scipy
%cd /path/to/your/directory/
!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install -r requirements.txt
!pip3 install transformers==4.19.0 einops==0.5.0 rotary_embedding_torch==0.1.5 unidecode==1.3.5
!python3 setup.py install
!pip3 install -U pydub
!pip install moviepy==2.0.0.dev2
!pip install imageio==2.25.1
!cat /etc/ImageMagick-6/policy.xml | sed 's/none/read,write/g'> /etc/ImageMagick-6/policy.xml
!apt install imagemagick
!pip install pysrt==1.1.2

In [ ]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio, load_voice, load_voices

tts = TextToSpeech()

In [ ]:
import os
import locale
import json
from pydub import AudioSegment
import re
import moviepy.editor as mp
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip
import re
import sys
import pysrt
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip


def training_voice(name, path):
  CUSTOM_VOICE_NAME = name
  custom_voice_folder = f"tortoise/voices/{CUSTOM_VOICE_NAME}" #change the path accordingly

  os.makedirs(custom_voice_folder, exist_ok=True)

  with open(path, 'rb') as f:
      file_data = f.read()
      with open(os.path.join(custom_voice_folder, f'{CUSTOM_VOICE_NAME}.wav'), 'wb') as new_file:
          new_file.write(file_data)

locale.getpreferredencoding = lambda: "UTF-8"

def GettingSpeakerAudio(audio_file, txt_file):
  def convert_to_wav(audio_segment, counter):
      filename = f"temp_{counter}.wav"
      audio_segment.export(filename, format="wav")
      return filename

  def segment_audio(audio_file, txt_file):
      audio = AudioSegment.from_file(audio_file, format="mp3")

      segments = {}
      with open(txt_file, 'r', encoding='utf-8') as file:
          txt_data = file.read().split('\n\n')
          counter = 0
          for data in txt_data:
              data = data.split('\n')
              speaker = data[2].split(']')[0][1:]
              if speaker.find('[') != -1:
                  continue

              transcript = data[2].split(']')[1]
              start_time = data[1].split(' --> ')[0]
              start_parts = start_time.replace(',', ':').split(':')
              end_time = data[1].split(' --> ')[1]
              end_parts = end_time.replace(',', ':').split(':')
              start_ms = int(start_parts[0]) * 60 * 60 * 1000 + int(start_parts[1]) * 60 * 1000 + float(
                  start_parts[2]) * 1000
              end_ms = int(end_parts[0]) * 60 * 60 * 1000 + int(end_parts[1]) * 60 * 1000 + float(end_parts[2]) * 1000
              audio_segment = audio[start_ms:end_ms + 1000]
              filename = convert_to_wav(audio_segment, counter)
              counter += 1
              audio_segment.export(filename, format="wav")
              if speaker not in segments:
                  segments[speaker] = []
              segments[speaker].append({"transcript": transcript, "filename": filename})
      return segments

  def combine_audio_for_speaker(segments, speaker):
      combined_audio = AudioSegment.empty()
      for segment in segments:
          audio_segment = AudioSegment.from_file(segment["filename"], format="wav")
          combined_audio += audio_segment
      filename = f"/content/{speaker}.wav" #change the path accordingly
      combined_audio.export(filename, format="wav")
      return filename


  audio_segments = segment_audio(audio_file, txt_file)

  output_json = json.dumps({"speakers": audio_segments}, indent=2)

  for speaker in audio_segments:
      f = combine_audio_for_speaker(audio_segments[speaker], speaker)
  return audio_segments

def mainThing(text,count, name):
  voice_samples, conditioning_latents = load_voice(name)
  gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents,
                            preset=preset)
  torchaudio.save(f'/content/generated-{name}{count}.wav', gen.squeeze(0).cpu(), 24000) #change the path accordingly

def generate_new_speech(audio_segments):
  for speaker in audio_segments:
      audio_path = f"/content/{speaker}.wav" #change the path accordingly
      training_voice(speaker, audio_path)
      print()
      print("Generating for ...", speaker)
      print()
      count =0
      for transcript in audio_segments[speaker]:
        text = transcript["transcript"]
        mainThing(text, count, speaker)
        count+=1

def segment_audio_new(audio_file, txt_file):
    count_another=0
    audio = AudioSegment.from_file(audio_file, format="mp3")
    speakers_count = {}
    segments = {}
    with open(txt_file, 'r', encoding='utf-8') as file:
        txt_data = file.read().split('\n\n')
        for data in txt_data:
            data = data.split('\n')
            current_speaker = data[2].split(']')[0][1:]
            if current_speaker.find('[') != -1:
                continue
            transcript = data[2].split(']')[1]
            start_time = data[1].split(' --> ')[0]
            start_parts = start_time.replace(',', ':').split(':')
            end_time = data[1].split(' --> ')[1]
            end_parts = end_time.replace(',', ':').split(':')
            start_ms = int(start_parts[0]) * 60 * 60 * 1000 + int(start_parts[1]) * 60 * 1000 + float(
                start_parts[2]) * 1000
            end_ms = int(end_parts[0]) * 60 * 60 * 1000 + int(end_parts[1]) * 60 * 1000 + float(end_parts[2]) * 1000
            try:
                speakers_count[current_speaker] += 1
            except KeyError:
                speakers_count[current_speaker] = 0

            print("Speaker:", current_speaker)
            print("Count:", speakers_count[current_speaker])
            path_now=f"/content/generated-{current_speaker}{speakers_count[current_speaker]}.wav" #change the path accordingly
            audio_now = AudioSegment.from_file(path_now, format="wav")
            audio_now.export(f"/content/{count_another}.wav", format="wav") #change the path accordingly
            count_another+=1



def full_combine_with_music(file_path, noise_path, output_path):
  def parse_annotations(file_path):
      with open(file_path, 'r', encoding='utf-8') as file:
          lines = file.readlines()

      annotations = []

      for line in lines:
          match = re.match(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})', line)
          if match:
              start_time, end_time = match.group(1), match.group(2)
              annotations.append({'start': time_to_ms(start_time), 'end': time_to_ms(end_time), 'lines': []})
          elif line.strip():
              match = re.match(r'\[([^\]]+)\](.*)', line)
              if match:
                  speaker, spoken_line = match.group(1), match.group(2).strip()
                  annotations[-1]['lines'].append({'speaker': speaker, 'line': spoken_line})

      return annotations

  def time_to_ms(time_str):
      m, s = map(int, re.split('[:,]', time_str)[1:3])
      return m * 60000 + s * 1000

  from pydub import AudioSegment

  def combine_audio_with_music(annotations, music_file):
      combined_audio = AudioSegment.silent()

      for i, annotation in enumerate(annotations, start=0):

            audio_file = f"/content/{i}.wav" #change the path accordingly

            individual_segment = AudioSegment.from_file(audio_file)

            gap_duration = annotation['start'] - len(combined_audio)

            if gap_duration > 0:
                gap = AudioSegment.silent(duration=gap_duration)
                combined_audio += gap

            combined_audio += individual_segment

      music = AudioSegment.from_file(music_file)

      combined_audio_with_music = combined_audio.overlay(music)

      return combined_audio_with_music


  def main(file_path,noise_path,output_path):
      annotations = parse_annotations(file_path)

      for i, annotation in enumerate(annotations, start=1):
          print(f"{i}. Start: {annotation['start']} ms, End: {annotation['end']} ms")
          for line in annotation['lines']:
              print(f"   [{line['speaker']}] {line['line']}")
      combined_audio = combine_audio_with_music(annotations, noise_path)
      combined_audio.export(output_path, format="wav")
  main(file_path,noise_path,output_path)



def video_srt_and_audio_add(video_path,audio_path,output_path,subtitle_path,output_srt_path):

  def remove_speaker_names(srt_content):
      pattern = re.compile(r'\[.*?\]')

      cleaned_subtitles = [re.sub(pattern, '', subtitle) for subtitle in srt_content]

      return cleaned_subtitles

  def process_srt(input_path, output_path):
      with open(input_path, 'r', encoding='utf-8') as file:
          srt_content = file.readlines()
      cleaned_subtitles = remove_speaker_names(srt_content)

      cleaned_subtitle_content = ''.join(cleaned_subtitles)

      with open(output_path, 'w', encoding='utf-8') as file:
          file.write(cleaned_subtitle_content)

  process_srt(subtitle_path, output_srt_path)
  audio = mp.AudioFileClip(audio_path)
  video = mp.VideoFileClip(video_path)

  final_video = video.set_audio(audio)
  final_video.write_videofile(output_path)

  def time_to_seconds(time_obj):
      return time_obj.hours * 3600 + time_obj.minutes * 60 + time_obj.seconds + time_obj.milliseconds / 1000


  def create_subtitle_clips(subtitles, videosize,fontsize=24, font='Arial', color='yellow', debug = False):
      subtitle_clips = []

      for subtitle in subtitles:
          start_time = time_to_seconds(subtitle.start)
          end_time = time_to_seconds(subtitle.end)
          duration = end_time - start_time

          video_width, video_height = videosize

          text_clip = TextClip(subtitle.text, fontsize=fontsize, font=font, color=color, bg_color = 'black',size=(video_width*3/4, None), method='caption').set_start(start_time).set_duration(duration)
          subtitle_x_position = 'center'
          subtitle_y_position = video_height* 4 / 5

          text_position = (subtitle_x_position, subtitle_y_position)
          subtitle_clips.append(text_clip.set_position(text_position))

      return subtitle_clips

  video = VideoFileClip(output_path)
  subtitles = pysrt.open(output_srt_path)

  begin,end= output_path.split(".mp4")
  output_video_file = begin+'_subtitled'+".mp4"

  print ("Output file name: ",output_video_file)

  subtitle_clips = create_subtitle_clips(subtitles,video.size)
  final_video = CompositeVideoClip([video] + subtitle_clips)
  final_video.write_videofile(output_video_file)

In [ ]:
subtitle_path = "path/to/your/file"
output_srt_path = "path/to/your/directory"
vocals="/path/to/your/directory"
subtitle_file="/path/to/your/directory"
noise_file="/path/to/your/directory"
output_file="path/to/your/directory"
output_file_final="/path/to/your/directory" # Final Output
video_path = "/path/to/your/directory/{" # Input Video


preset = "ultra_fast"

audio_segments = GettingSpeakerAudio(vocals,subtitle_file)

generate_new_speech(audio_segments)

segment_audio_new(vocals, subtitle_file)

full_combine_with_music(subtitle_file, noise_file, output_file_final)

video_srt_and_audio_add(video_path,output_file,output_file_final,subtitle_path,output_srt_path)
